In [1]:
%cd ..

/Users/tomtalpir/Random/pyERGM


In [2]:
from utils import *
from ergm import ERGM

In [3]:
n_nodes = 10

stats_calculator = NetworkStatistics(metric_names=["num_edges", "num_triangles"])

ergm = ERGM(n_nodes, stats_calculator, is_directed=False)

thetas = [-np.log(2), np.log(3)]

ergm.fit(precalculated_thetas=thetas)

In [10]:
sampled_net = ergm.sample_network(steps=30)
print(sampled_net)

proba = ergm.calculate_probability(sampled_net)
print(proba)

[[0. 1. 0. 1. 1. 1. 1. 0. 0. 0.]
 [1. 0. 1. 1. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 1. 0. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 1. 1. 0. 0.]]
3.341060761292946e-05
